# FAST-ODD - Multirotor Design

FAST is a framework for performing rapid Overall Aircraft Design. The computational core of FAST is based on the  [OpenMDAO framework](https://openmdao.org/).

FAST-ODD is the drone declination of FAST.

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes a multirotor drone. In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import openmdao.api as om
import logging
import shutil
import fastoad.api as oad

DATA_FOLDER_PATH = 'data'

WORK_FOLDER_PATH = 'workdir'

CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, 'odd_process_multirotor.yaml')
SOURCE_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In the configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the reference parameters from my_ref_values.xml as default values:

In [2]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Loaded variable descriptions in D:\THESE\Tools\FAST-ODD\data\../models
INFO    : Loaded variable descriptions in D:\THESE\Tools\FAST-ODD\data\../models
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models
INFO    : Installed bundle models (ID 199 )
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models/Energy_source
INFO    : Installed bundle Energy_source (ID 200 )
INFO    : Installed bundle Energy_source.energy_source (ID 202 )
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models/ESC
INFO    : Installed bundle ESC.esc (ID 205 )
INFO    : Installed bundle ESC (ID 203 )
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models/Missions
INFO    : Installed bundle Missions.mission_design (ID 208 )
INFO    : Installed bundle Missions (ID 206 )
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models/Motor
INFO    : Installed bundle Motor (ID 209 )
INFO    : Installed bundle Motor.motor (ID 211 )
INFO    : 

'D:\\THESE\\Tools\\FAST-ODD\\workdir\\problem_inputs.xml'

Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [3]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : N2 diagram written in D:\THESE\Tools\FAST-ODD\workdir\n2.html


## 2. Inputs

The problem inputs file contains 3 groups:
- specifications: these are the top-level requirements such as maximum thrust-to-weight ratio and the design mission definition (hover duration, climb height, range and maximum payload);
- data: everything related to the components and system architecture.
- addons: here you will find parameters which are specific to an application. For the "safety" add-on you can specify a design mission (2 routes plus a diversion) and a maxiumum failure sizing factor for the thrust.

*Note:
Further work should focus on a better way to handle the mission definition. For now, if you work with the "safety" add-on, the design optimization takes account for both the mission defined in "specifications" and the one defined in "addons" (see the constraints section in the configuration .yaml file). Thus, it is recommended (but not mandatory) to work only with the parameters of the "addons" group. Only the maximum payload defined in the "specifications" group should be modified, as it is used to size the propulsion chain.*

In [20]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')
oad.variable_viewer(INPUT_FILE)

In [21]:
#eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

## 3. Multirotor MDO

In [22]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loaded variable descriptions in D:\THESE\Tools\FAST-ODD\data\../models
INFO    : Loaded variable descriptions in D:\THESE\Tools\FAST-ODD\data\../models
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models
INFO    : Installed bundle models (ID 199 )
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models/Energy_source
INFO    : Installed bundle Energy_source (ID 200 )
INFO    : Installed bundle Energy_source.energy_source (ID 201 )
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models/ESC
INFO    : Installed bundle ESC (ID 203 )
INFO    : Installed bundle ESC.esc (ID 204 )
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models/Missions
INFO    : Installed bundle Missions.mission_design (ID 207 )
INFO    : Installed bundle Missions (ID 206 )
INFO    : Loading bundles from D:\THESE\Tools\FAST-ODD\data\../models/Motor
INFO    : Installed bundle Motor.motor (ID 210 )
INFO    : Installed bundle Motor (ID 209 )
INFO    : 

Optimization terminated successfully    (Exit mode 0)
            Current function value: [8.80987654]
            Iterations: 11
            Function evaluations: 12
            Gradient evaluations: 11
Optimization Complete
-----------------------------------


INFO    : Computation finished after 3.99 seconds
INFO    : Problem outputs written in D:\THESE\Tools\FAST-ODD\workdir\problem_outputs.xml


In [23]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
# MDO_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mdo.xml')
MDO_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_mdo.xml')
shutil.copy(OUTPUT_FILE, MDO_OUTPUT_FILE)
oad.optimization_viewer(CONFIGURATION_FILE)

In [24]:
oad.variable_viewer(OUTPUT_FILE)

## 4. Analysis and plots

In [25]:
from utils.postprocessing.analysis_and_plots import *

#fig = drone_geometry_plot(MDA_OUTPUT_FILE, name='Drone MDA')
#fig = drone_geometry_plot(MDO_OUTPUT_FILE, name='Drone MDO', fig=fig)

fig = drone_geometry_plot(MDO_OUTPUT_FILE, name='Drone MDO')
fig.show()

In [26]:
fig = mass_breakdown_sun_plot_drone(OUTPUT_FILE)
fig.show()

In [27]:
#OUTPUT_FILE_1 = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Cd_0_4.xml')
#OUTPUT_FILE_2 = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Cd_0_8.xml')
#OUTPUT_FILE_3 = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Cd_1_2.xml')

#fig = mass_breakdown_bar_plot_drone(OUTPUT_FILE_1, name='Cd = 0.4')
#fig = mass_breakdown_bar_plot_drone(OUTPUT_FILE_2, name='Cd = 0.8', fig=fig)
#fig = mass_breakdown_bar_plot_drone(OUTPUT_FILE_3, name='Cd = 1.2', fig=fig)

fig = mass_breakdown_bar_plot_drone(MDO_OUTPUT_FILE, name='Drone MDO')
# fig = mass_breakdown_bar_plot_drone(MDO_OUTPUT_FILE, name='Drone MDO', fig=fig)
fig.show()
plotly.io.write_image(fig, 'output_file.pdf', format='pdf')

In [28]:
# Design mission
fig = energy_breakdown_sun_plot_drone(OUTPUT_FILE)
fig.show()